# Credit Card Fraud

Fraud activities are considered uncommon or outliers transactions, which is probably one of the main characteristics regarding Fraud. As the authors of the book Fraud Analytics using Descriptive, Predictive, and Social Network Techniques: A Guide to Data Science for Fraud Detection pointed out:

“This makes it difficult [because of the outlier characteristic] to both detect fraud, since the fraudulent cases are covered by the non-fraudulent ones, and to learn from historical cases to build a powerful fraud-detection system since only few examples are available”

Hence, it is imperative to overcome this issue considering the unbalanced nature of the data.

In this project, we will use the credit card data available in [https://www.kaggle.com/mlg-ulb/creditcardfraud](https://www.kaggle.com/mlg-ulb/creditcardfraud). Moreover, we will use PySpark, an Interface for Apache Spark in Python, since it is an excellent tool dealing with Big Data.

### Feature Technicalities:

- PCA Transformation: The description of the data says that all the features went through a PCA transformation (Dimensionality Reduction technique) (Except for time and amount).
- Scaling: In order to implement a PCA transformation features need to be previously scaled. (In this case, all the V features have been scaled)


# EDA - Exploratory Data Analysis

In [2]:
# Import libraries
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

# Import PySpark libraries
from pyspark.sql import Window
import pyspark.sql.types as t
import pyspark.sql.functions as f
from pyspark.sql import SparkSession


# Configure Spark Session
spark = SparkSession.builder \
    .appName("Credit Card Fraud") \
    .master("local[*]") \
    .config("spark.driver.bindAddress", '127.0.0.1') \
    .getOrCreate()
sc = spark.sparkContext


23/05/05 06:31:43 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

In [3]:
# Load file
df = spark.read.csv('datasets/creditcard.csv', header=True, inferSchema=True, sep=",")
# Print Schema
df.printSchema()

root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

In [4]:
# Spark DataFrame Shape
print(f"Number of columns: {len(df.columns)}")
print(f"Number of Records: {df.count()}")

Number of columns: 31


Number of Records: 284807


The dataset contains **284,807 records** and consists of **31 columns**. Out of these columns, **28 columns (V1 to V28)** are results of the **Principal Component Analysis (PCA)** technique. These columns are likely to be more abstract in nature since they are generated from linear combinations of the original features. Therefore, it may be difficult to interpret the individual contributions of these features towards the prediction task.

On the other hand, the remaining three columns are:

- **Time**: This column indicates the time elapsed in seconds between each transaction and the first transaction in the dataset.
- **Amount**: This column indicates the transaction amount.
- **Class**: This column indicates the fraud status of the transaction. The value 1 indicates fraud, and 0 indicates a legitimate transaction.

It is important to note that since the **PCA** technique is used, the original feature names and descriptions are not available. Therefore, feature engineering might be necessary to extract meaningful features from the given dataset.


In [5]:
# Inspecting the first 10 records
df.limit(10).toPandas()

23/05/02 21:54:09 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [6]:
#statistics
df.toPandas().describe().T

,count,mean,std,min,25%,50%,75%,max
Time,284807.0,9.481386e+04,47488.145955,0.000000,54201.500000,84692.000000,139320.500000,172792.000000
V1,284807.0,1.168375e-15,1.958696,-56.407510,-0.920373,0.018109,1.315642,2.454930
V2,284807.0,3.416908e-16,1.651309,-72.715728,-0.598550,0.065486,0.803724,22.057729
V3,284807.0,-1.379537e-15,1.516255,-48.325589,-0.890365,0.179846,1.027196,9.382558
V4,284807.0,2.074095e-15,1.415869,-5.683171,-0.848640,-0.019847,0.743341,16.875344
V5,284807.0,9.604066e-16,1.380247,-113.743307,-0.691597,-0.054336,0.611926,34.801666
V6,284807.0,1.487313e-15,1.332271,-26.160506,-0.768296,-0.274187,0.398565,73.301626
V7,284807.0,-5.556467e-16,1.237094,-43.557242,-0.554076,0.040103,0.570436,120.589494
V8,284807.0,1.205498e-16,1.194353,-73.216718,-0.208630,0.022358,0.327346,20.007208
V9,284807.0,-2.406306e-15,1.098632,-13.434066,-0.643098,-0.051429,0.597139,15.594995


The dataset contains a total of **284807 records**. The **time column** in the dataset indicates the time elapsed in seconds between each transaction and the first transaction in the dataset. The maximum time recorded between transactions is **172,792 seconds**. This is equivalent to 2880 minutes or approximately **48 hours**. Since the timeframe of the dataset is only 2 days, it means that the last transaction was made two days after the first one.

The amount column indicates the transaction amount. The biggest transaction recorded in the dataset is worth **$25,691.16**.


## Checking Missing Values

In [ ]:
#Check missing and null data
df.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

Upon checking the dataset, it was observed that there are no missing values present.
Each column was inspected for null or NaN values, and it was found that the dataset is complete!


## 1.2 Univariate Analysis

### 1.2.1 Class

#### Q1. HOW MANY ROWS IN THE DATASET REPRESENT CREDIT CARD FRAUD?

In [5]:
#Number of Frauds and non-frauds
classFreq = df.groupBy("Class").count()
classFreq.show()

+-----+------+
|Class| count|
+-----+------+
|    1|   492|
|    0|284315|
+-----+------+



In [6]:
total = classFreq.select("count").agg({"count": "sum"}).collect().pop()['sum(count)']
result = classFreq.withColumn('percent', f.format_number(classFreq['count']/total * 100, 2))
result.show()

+-----+------+-------+
|Class| count|percent|
+-----+------+-------+
|    1|   492|   0.17|
|    0|284315|  99.83|
+-----+------+-------+



Out of the total of **284,807 records**, only **492** represent credit card fraud, which is approximately **0.17%** of the dataset. Therefore, we have an imbalanced dataset.

In [ ]:
# Create a pandas dataframe with only the Class column
fraud_df = df.select("Class").toPandas()

# Count the number of transactions for each class (fraudulent or not)
fraud_counts = fraud_df['Class'].value_counts()

# Calculate the percentage and absolute number of each class
fraud_perc = fraud_counts / fraud_counts.sum() * 100
fraud_abs = fraud_counts.values

# Create a bar plot with logarithmic y-axis
sns.barplot(x=fraud_counts.index, y=fraud_counts.values)
plt.yscale('log')

# Add percentage and absolute value labels to the bars
for i, v in enumerate(fraud_counts):
    plt.text(i, v, f"{fraud_perc[i]:.1f}% ({fraud_abs[i]:,})", ha='center', va='bottom', fontweight='bold')

# Set the y-axis labels and formatter
plt.ylabel('Number of Transactions')
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:.0f} K'.format(x/1000)))

# Set the axis labels and title
plt.xlabel('Transaction Class')
plt.xticks([0, 1], ['Non-Fraud', 'Fraud'])
plt.title('Distribution of fraudulent transactions')

plt.show()

As mentioned earlier, the majority of transactions are legitimate. If we use this dataset as the foundation for our predictive models and analysis, we may encounter many errors, and our algorithms are likely to overfit since they will "presume" that most transactions are not fraudulent. However, we do not want our model to presume; we want our model to identify patterns that indicate fraud.



### 1.2.2 Amount -- Should we remove this? At least for me, doesn't make much sense.

In [ ]:
# create a function to group time values
@f.udf(returnType=t.StringType())
def time_udf(time):
    if time < 50000:
        return "Under 50K s"
    elif 50000 <= time <= 100000:
        return "Between 50K and 100K s"
    elif time > 100000:
        return "Over 100K s"
    else:
        return "NA"

# apply the function to the "Time" column
df = df.withColumn('time_udf', time_udf('Time'))

df.limit(10).toPandas()

In [ ]:
# Define window function with partitionBy clause
window = Window.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

# Create time group table
time_group_table = df.select("time_udf", "Amount"). \
    groupBy("time_udf"). \
    agg(
    f.count("Amount").alias("UserCount"),
    f.mean("Amount").alias("Amount_Avg"),
    f.min("Amount").alias("Amount_Min"),
    f.max("Amount").alias("Amount_Max")). \
    withColumn("total", f.sum("UserCount").over(window)). \
    withColumn("Percent", f.round(f.col("UserCount")*100 / f.col("total"), 2)). \
    drop("total"). \
    orderBy(f.desc("Percent"))

time_group_table.limit(10).toPandas()


In [ ]:
# Convert PySpark dataframe to Pandas dataframe
time_group_df = time_group_table.toPandas()

# Create barplot
ax = sns.barplot(x="time_udf", y="Percent", data=time_group_df)

for p in ax.patches:
    abs_value = p.get_height()
    ax.annotate(f'{abs_value}',
                (p.get_x() + p.get_width() / 2., abs_value),
                ha='center', va='center',
                xytext=(0, 9),
                textcoords='offset points')

plt.title("User Count and Transaction Amount by Time Group")
plt.xlabel("Time Group")
plt.ylabel("Percentage")
plt.show();

In [ ]:
# All Transactions
df_aux = df.select("Class", "Amount").toPandas()

# Define amount ranges
amount_ranges = [
    {"range": "Transaction Value <= $100", "min_amount": 0, "max_amount": 100},
    {"range": "Transaction Value between \$101 and \$2000", "min_amount": 101, "max_amount": 2000},
    {"range": "Transaction Value between \$2001 and \$5000", "min_amount": 2001, "max_amount": 5000},
    {"range": "Transaction Value > $5000", "min_amount": 5001, "max_amount": df_aux["Amount"].max()}
]

# Create four subplots for different amount ranges
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

for idx, amount_range in enumerate(amount_ranges):
    row_idx = idx // 2
    col_idx = idx % 2

    # Filter transactions within amount range
    df_range = df_aux[(df_aux["Amount"] > amount_range["min_amount"]) & (df_aux["Amount"] <= amount_range["max_amount"])]

    # Plot histogram
    sns.histplot(data=df_range, x="Amount", ax=axes[row_idx, col_idx], hue="Class", kde=True)
    axes[row_idx, col_idx].set_title(amount_range["range"])
    axes[row_idx, col_idx].set_ylabel("Number of Transactions")
    axes[row_idx, col_idx].legend(labels=["Fraud", "Non-Fraud"])

fig.suptitle("All Transactions", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Only fraud transactions
only_fraud = df.filter(df.Class == 1).select("Amount").toPandas()

# Add a pallet
aux_pal = ["#ff7f7f", "#ff3c3c"]

# Create three subplots for different amount ranges
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Amount range <= 100
sns.histplot(data=only_fraud[only_fraud["Amount"] <= 100], x="Amount", ax=axes[0], kde=True, color=aux_pal[0])
axes[0].set_title("Transaction Amount <= $100")
axes[0].set_ylabel("Number of Transactions")

# Amount range > 100
sns.histplot(data=only_fraud[(only_fraud["Amount"] > 100)], x="Amount", ax=axes[1], kde=True, color=aux_pal[1])
axes[1].set_title("Transaction Amount > $100")
axes[1].set_ylabel("Number of Transactions")

fig.suptitle("Only Fraudulent Transactions", fontsize=16)
plt.tight_layout()
plt.show()

### 1.2.3 Remaining variables

In [ ]:
# create several histograms to show the distributions of the features
fig, axs = plt.subplots(8, 4, figsize=(25, 15))
fig.suptitle("Distribution of Features", fontsize=14)

for col, ax in zip(df.columns, axs.flatten()):
    ax.hist(df.select(col).toPandas()[col])
    ax.grid(False)
    ax.tick_params(axis='x', labelrotation=45, labelsize=14)
    ax.tick_params(axis='y', labelsize=15)
    ax.set_title(col.upper(), fontsize=14)

plt.tight_layout()
plt.subplots_adjust(top=0.9, hspace=0.4)
plt.show()

Through the histograms, we can observe that the feature "Time" does not follow a normal distribution, with a lower occurrence in the range of 75,000-100,000 seconds. On the other hand, the features Vx show a normalized distribution centered around the value 0, with some negative values.

## 1.3 Imbalanced Data

To address the imbalanced data issue, we will implement the technique of Random Under Sampling. The aim is to balance the dataset by removing data, which helps to avoid model overfitting.

The following steps will be taken:

    - Determine the degree of imbalance in the class by using the "value_counts()" function on the class column to count the number of instances for each label.

    - Bring the number of non-fraud transactions to the same amount as fraud transactions (assuming we want a 50/50 ratio),which will be equivalent to 492 cases of fraud and 492 cases of non-fraud transactions.

    - Shuffle the data to ensure that our models can maintain a certain accuracy every time we run the script.

**Note**: It is important to note that the Random Under Sampling technique may result in a loss of information since we are reducing the number of instances in the dataset.

In [7]:
# Select fraud and non-fraud transactions and limit non-fraud transactions to the same number as fraud transactions
fraud_df = df.filter(f.col('Class') == 1)
non_fraud_df = df.filter(f.col('Class') == 0).limit(fraud_df.count())

# Combine fraud and non-fraud transactions and shuffle the data
balanced_df = fraud_df.union(non_fraud_df).orderBy(f.rand())

# Show 10 rows of the shuffled, balanced dataframe
balanced_df.limit(10).toPandas()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,339.0,1.202783,-1.464842,0.703284,-1.462520,-1.462982,0.495419,-1.368709,0.236519,-1.938471,...,0.104264,0.575825,-0.033504,-0.278811,0.145526,-0.071895,0.067936,0.026132,98.00,0
1,218.0,-0.391725,1.106355,0.787765,0.510525,0.832968,0.151659,0.588155,0.312914,-0.772829,...,0.122192,0.365018,-0.213950,-0.421854,0.045355,-0.323299,0.080659,0.037889,1.00,0
2,68207.0,-13.192671,12.785971,-9.906650,3.320337,-4.801176,5.760059,-18.750889,-37.353443,-0.391540,...,27.202839,-8.887017,5.303607,-0.639435,0.263203,-0.108877,1.269566,0.939407,1.00,1
3,67.0,-1.494668,0.837241,2.628211,3.145414,-0.609098,0.258495,-0.012189,0.102136,-0.286164,...,-0.140047,0.355044,0.332720,0.718193,-0.219366,0.118927,-0.317486,-0.340783,28.28,0
4,17520.0,-5.268053,9.067613,-15.960728,10.296603,-4.708241,-3.395375,-11.161057,5.499963,-5.667376,...,2.004110,0.191058,0.622928,-1.209264,-0.374799,0.648798,1.584697,0.720056,1.00,1
5,290.0,-0.695818,0.581773,2.378180,0.063396,0.329119,-0.449865,1.269104,-0.758363,0.381712,...,-0.327948,-0.369683,-0.426987,0.420170,0.235207,0.194957,-0.708471,-0.621219,30.50,0
6,304.0,1.049639,0.066437,0.059213,0.285469,0.635754,1.473074,-0.376993,0.579045,-0.182240,...,-0.141137,-0.304712,0.206105,-1.397547,-0.093121,0.227115,0.042340,0.001972,1.98,0
7,154.0,1.315601,0.413148,-0.018953,0.487834,0.132331,-0.574307,0.146563,-0.215923,-0.080276,...,-0.329567,-0.891188,0.011352,-0.477579,0.365667,0.143254,-0.016757,0.024640,1.98,0
8,93853.0,-5.839192,7.151532,-12.816760,7.031115,-9.651272,-2.938427,-11.543207,4.843627,-3.494276,...,2.462056,1.054865,0.530481,0.472670,-0.275998,0.282435,0.104886,0.254417,316.06,1
9,343.0,1.004760,0.423992,0.579929,2.476754,-0.124375,-0.363936,0.297698,-0.104342,-1.231239,...,0.128929,0.195254,-0.119654,0.557573,0.516518,-0.004221,-0.028055,0.025729,75.83,0


In [10]:
aux = balanced_df.toPandas()
aux.to_csv('datasets/dfscaled.csv', index=False)

In [11]:
aux.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,339.0,1.202783,-1.464842,0.703284,-1.462520,-1.462982,0.495419,-1.368709,0.236519,-1.938471,...,0.104264,0.575825,-0.033504,-0.278811,0.145526,-0.071895,0.067936,0.026132,98.00,0
1,218.0,-0.391725,1.106355,0.787765,0.510525,0.832968,0.151659,0.588155,0.312914,-0.772829,...,0.122192,0.365018,-0.213950,-0.421854,0.045355,-0.323299,0.080659,0.037889,1.00,0
2,68207.0,-13.192671,12.785971,-9.906650,3.320337,-4.801176,5.760059,-18.750889,-37.353443,-0.391540,...,27.202839,-8.887017,5.303607,-0.639435,0.263203,-0.108877,1.269566,0.939407,1.00,1
3,67.0,-1.494668,0.837241,2.628211,3.145414,-0.609098,0.258495,-0.012189,0.102136,-0.286164,...,-0.140047,0.355044,0.332720,0.718193,-0.219366,0.118927,-0.317486,-0.340783,28.28,0
4,17520.0,-5.268053,9.067613,-15.960728,10.296603,-4.708241,-3.395375,-11.161057,5.499963,-5.667376,...,2.004110,0.191058,0.622928,-1.209264,-0.374799,0.648798,1.584697,0.720056,1.00,1


In [ ]:
# Convert the Spark dataframe to a pandas dataframe
fraud_df = balanced_df.select("Class").toPandas()

# Create a countplot with logarithmic y-axis
sns.countplot(x='Class', data=fraud_df, palette='Set1', order=[1, 0])

# Calculate the percentage of each class
fraud_counts = fraud_df['Class'].value_counts()
fraud_perc = fraud_counts / fraud_counts.sum() * 100

# Set the y-axis labels and formatter
plt.ylabel('Number of Transactions')

# Set the axis labels and title
plt.xlabel('Transaction Class')
plt.xticks([0, 1], ['Fraud', 'Non-Fraud'])
plt.title('Balanced Distribution of Transactions')

plt.show()

Now that we have our dataframe correctly balanced, we can go further with our analysis and data preprocessing.


## Correlations

Correlation matrices are the essence of understanding our data. We want to know if there are features that influence heavily in whether a specific transaction is a fraud. However, it is important that we use the correct dataframe (Balanced) in order for us to see which features have a high positive or negative correlation in regard to fraud transactions.

Summary and Explanation:

    Negative Correlations: V17, V14, V12 and V10 are negatively correlated. Notice how the lower these values are, the more likely the end result will be a fraud transaction.
    Positive Correlations: V2, V4, V11, and V19 are positively correlated. Notice how the higher these values are, the more likely the end result will be a fraud transaction.
    BoxPlots: We will use boxplots to have a better understanding of the distribution of these features in fradulent and non fradulent transactions.

Note: We have to make sure we use the subsample in our correlation matrix or else our correlation matrix will be affected by the high imbalance between our classes. This occurs due to the high class imbalance in the original dataframe.


In [ ]:
df = df.drop("time_udf")
balanced_df = balanced_df.drop("time_udf")

plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
sns.heatmap(df.toPandas().corr(), cmap='coolwarm_r')
plt.title('Correlations with Unbalanced Data')

plt.subplot(1, 2, 2)
sns.heatmap(balanced_df.toPandas().corr(), cmap='coolwarm_r')
plt.title('Correlations with Balanced Data')

plt.show()

In [ ]:
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
sns.heatmap(balanced_df.toPandas().corr() > 0.7,cbar=False, annot_kws={'size':20})
plt.title('Correlations with Balanced Data > 0.7')

plt.subplot(1, 2, 2)
sns.heatmap(balanced_df.toPandas().corr() < -0.7,cbar=False, annot_kws={'size':20})
plt.title('Correlations with Balanced Data < -0.7')

plt.show();


#### Conclusions from Correlation Matrix
Based on our correlation matrix, we have concluded the following:

- The variables that were a product of PCA are not correlated with each other, meaning they are independent.
- The variable "Time" seems to be negatively correlated with all of the "Vx" variables, which means that as "Time" increases, the values of "Vx" decrease, and vice versa.
- The variables "Time" and "Class" seem to have no correlation, indicating that the time at which a transaction occurs has no influence on whether it is fraudulent or not.
- The variable "Class" seems to be negatively correlated with some of the "Vx" variables, indicating that certain values of "Vx" are more likely to be associated with fraudulent transactions, while not correlated at all with others.


## Distributions: Univariate

In [ ]:
# select all columns except the ones to exclude
cols_to_include = [col for col in balanced_df.columns if col not in ['Time']]
df_to_plot = balanced_df.select(*cols_to_include)

for col in df_to_plot.columns:
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    # plot histogram
    axs[0].hist(df_to_plot.select(col).rdd.flatMap(lambda x: x).collect(), bins=50)
    axs[0].set_title(f"Distribution of {col} values", size=14)
    axs[0].set_xlabel(col, size=12)
    axs[0].set_ylabel("Count", size=12)

    # plot boxplot
    sns.boxplot(data=df_to_plot.select(col).toPandas(), x=col, ax=axs[1])
    axs[1].set_title(f"{col} Boxplot", size=14)
    axs[1].set_xlabel(col, size=12)
    axs[1].set_ylabel("Count", size=12)

plt.show();

## Distributions: Bivariate

In [ ]:
#Bivariate Analysis

data = df_to_plot.toPandas()
fig, axes = plt.subplots(ncols=4, figsize=(20,4))

# Negative Correlations with our Class (The lower our feature value the more likely it will be a fraud transaction)
sns.boxplot(x="Class", y="V14", data=data,  ax=axes[0])
axes[0].set_title('V14 vs Class Negative Correlation')

sns.boxplot(x="Class", y="V12", data=data,  ax=axes[1])
axes[1].set_title('V12 vs Class Negative Correlation')


sns.boxplot(x="Class", y="V10", data=data, ax=axes[2])
axes[2].set_title('V10 vs Class Negative Correlation')


sns.boxplot(x="Class", y="V16", data=data, ax=axes[3])
axes[3].set_title('V16 vs Class Negative Correlation')

plt.show()

In [ ]:
fig, axes = plt.subplots(ncols=4, figsize=(20,4))

# Positive correlations (The higher the feature the probability increases that it will be a fraud transaction)
sns.boxplot(x="Class", y="V4", data=data,  ax=axes[0])
axes[0].set_title('V4 vs Class Positive Correlation')

sns.boxplot(x="Class", y="V11", data=data,  ax=axes[1])
axes[1].set_title('V11 vs Class Positive Correlation')


sns.boxplot(x="Class", y="V2", data=data, ax=axes[2])
axes[2].set_title('V2 vs Class Positive Correlation')


sns.boxplot(x="Class", y="V19", data=data, ax=axes[3])
axes[3].set_title('V19 vs Class Positive Correlation')

plt.show()

In [ ]:
hist_fraud = df_to_plot.filter(f.col('Class') == 1).drop('Class').toPandas()

hist_fraud.hist(bins=30, figsize=(10, 10))
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.8)
plt.show()

# Descriptive Problem
### 1.1 At what time of the day (hour) do most of fraudulent transactions occur?

In [ ]:
## Function to create time of day to be used by UDF (user defined function)
def create_time_of_day(second):
    current_date_time = str(datetime.datetime(2013, 9, 1, 0, 0, 0) + datetime.timedelta(seconds = second))
    time_of_day = current_date_time[11:13]
    return time_of_day

# Create a UDF
udf_time_of_day = f.udf(lambda x: create_time_of_day(x))

# Total amount of items
## Convert 'Time' column datatype to 'int'
df = df.withColumn("Time", df["Time"].cast(t.IntegerType()))

## Create a new column "TimeOfTheDay" using UDF
df = df.withColumn("TimeOfTheDay", udf_time_of_day("Time"))

## Convert 'Time' column datatype back to 'double'
df = df.withColumn("Time", df["Time"].cast(t.DoubleType()))

## Filter the dataframe for items customers did not purchase ('fraudulent')
df_filtered_fraudulent = df.where("Class == 1")

## Select the most items customers did not purchase ('fraudulent') and check the time of the day
df_filtered_fraudulent_time_of_day_count = df_filtered_fraudulent.groupBy("TimeOfTheDay") \
    .agg({"Class":"count"}) \
    .orderBy("TimeOfTheDay") \
    .withColumnRenamed("count(Class)", "NumberOfFrauds")
df_filtered_fraudulent_time_of_day_count_pd = df_filtered_fraudulent_time_of_day_count.toPandas()

## Visualize the most items customers did not purchase ('fraudulent') and check the time of the day (hour)
plt.figure(figsize = (10, 4), dpi = 100)
plt.plot(df_filtered_fraudulent_time_of_day_count_pd.TimeOfTheDay, df_filtered_fraudulent_time_of_day_count_pd.NumberOfFrauds)
plt.title("At what time of the day (hour) do most of fraudulent transactions occur?")
plt.xticks(rotation=30, ha='right')
plt.xlabel('Time of the Day (Hour)')
plt.ylabel('Number of Frauds')
plt.show()

### 1.2 At what time of the day (hour) are customers charged more for items that they did not purchase?

In [ ]:
# Total amount
## Select the most items charged customers did not purchase ('fraudulent') and check the time of the day (hour)
df_filtered_fraudulent_time_of_day_amount = df_filtered_fraudulent.groupBy("TimeOfTheDay") \
    .agg({"Amount":"sum"}) \
    .orderBy("TimeOfTheDay") \
    .withColumnRenamed("sum(Amount)", "AmountChargedOfFrauds")
df_filtered_fraudulent_time_of_day_amount_pd = df_filtered_fraudulent_time_of_day_amount.toPandas()

## Visualize the most items charged customers did not purchase ('fraudulent') and check the time of the day (hour)
plt.figure(figsize = (10, 4), dpi = 100)
plt.plot(df_filtered_fraudulent_time_of_day_amount_pd.TimeOfTheDay, df_filtered_fraudulent_time_of_day_amount_pd.AmountChargedOfFrauds)
plt.title("At what time of the day (hour) are customers charged more for items that they did not purchase?")
plt.xticks(rotation=30, ha='right')
plt.xlabel('Time of the Day (Hour)')
plt.ylabel('Amount Charged of Frauds')
plt.grid(False)
plt.show()